<a href="https://colab.research.google.com/github/nel-eleven11/Lab9_IA/blob/main/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Laboratorio 9

## Task 1 - Teoría

Responda las siguientes preguntas de forma clara y concisa, pueden subir un PDF bien dentro del mismo Jupyter Notebook.

1. Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models
2. Investigue qué son los factorial HMM (Hidden Markov Models)
3. Especifique en sus propias palabras el algoritmo Forward Backward para HMM
4. En el algoritmo de Forward Backward, por qué es necesario el paso de backward (puede escribir ejemploso casos para responder esta pregunta)

## Task 2 - Algoritmo Forward Backward en HMM

El algoritmo forward-backward se basa en una forma de programación dinámica.Para este task deberán investigar sobre el mismo. En este ejercicio estamos ante un modelo meteorológico representado por un Modelo Oculto de Markov (HMM) con dos estados: "Soleado" y "Lluvioso". Queremos predecir el tiempo en un día determinado basándonos en las observaciones de si el día anterior estuvo soleado o lluvioso. Para ello considere el siguiente código como una guía:

Con esto, considere lo siguiente:
1. Defina los parámetros del modelo oculto de Markov (HMM), incluidos estados, observaciones,
probabilidades iniciales, probabilidades de transición y probabilidades de emisión.
2. Cree una instancia de la clase HMM con los parámetros definidos.
3. Genere una secuencia de observaciones utilizando el método generate_sequence. Este paso es opcional
pero útil para realizar pruebas.
4. Utilice el método forward para calcular las probabilidades directas, que representan la probabilidad de
observar una secuencia particular de observaciones hasta un paso de tiempo específico y estar en un estado
particular en ese paso de tiempo.
5. Utilice el método backward para calcular las probabilidades hacia atrás, que representan la probabilidad de
observar la secuencia restante desde un paso de tiempo específico hasta el final de la secuencia, dado que
el sistema se encuentra en un estado particular en ese paso de tiempo.
6. Utilice el método compute_state_probabilities para calcular las probabilidades de estar en cada
estado en cada paso de tiempo dada la secuencia observada. Este paso implica combinar las probabilidades
hacia adelante y hacia atrás.
7. Imprima o analice las probabilidades calculadas según sea necesario.